Universidad Central de Venezuela  
Facultad de Ciencias  
Escuela de Computación  
Minería de Datos  

Nombre: Oscary Arocha  
C.I: 30.697.617 

<center><h1>Asignación #1: Pre-procesamiento</h1></center> 

Para esta asiganción se usará la librería pandas para preprocesar el dataset World University Rankings 2023.

Lo primero es cargar el dataset y estandarizar los valores nulos que se encontraron.

In [100]:
import pandas as pd
import numpy as np

# Definir valores que deben ser tratados como nulos
valores_nulos = ["nan", "NaN", "null", "n/a", "N/A",'—', '-']

df = pd.read_csv('./WorldUniversityRankings2023.csv', na_values=valores_nulos)
df.head()



,University Rank,Name of University,Location,No of student,No of student per staff,International Student,Female:Male Ratio,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,48 : 52,96.4,92.3,99.7,99.0,74.9,96.2
1,2,Harvard University,United States,"21,887",9.6,25%,50 : 50,95.2,94.8,99.0,99.3,49.5,80.5
2,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,47 : 53,94.8,90.9,99.5,97.0,54.2,95.8
3,3,Stanford University,United States,"16,164",7.1,24%,46 : 54,94.8,94.2,96.7,99.8,65.0,79.8
4,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,40 : 60,94.2,90.7,93.6,99.8,90.9,89.3


Como primer problema funcional, hay universidades que no tienen información en campos importantes como el ranking, el número de estudiantes y los ratios, incluso hay algunos registros sin nombre de la universidad, lo que nos deja sin información útil para el análisis por lo que se procede a eliminarlas para ahorrar recursos.

In [101]:
# Cantidad de registros antes de limpiar
print(f"Total de registros antes de limpiar: {len(df)} \n")

# Mostrar filas con NaN en 'University Rank'
nulos = df.query("`University Rank`.isna()")
print(nulos[['University Rank', 'Name of University']])

# Calcular cuántas filas tienen NaN en 'University Rank'
total_nulos = df['University Rank'].isna().sum()
print(f"\nHay {total_nulos} filas con NaN en la columna 'University Rank'")

# Eliminar filas nulas de 'University Rank' 
df = df.dropna(subset=['University Rank']) 

# Verificar cuántos registros quedaron
print(f"Total de registros después de limpiar: {len(df)}")


Total de registros antes de limpiar: 2341 

     University Rank                   Name of University
2209             NaN                                  NaN
2210             NaN                                  NaN
2211             NaN                                  NaN
2212             NaN                                  NaN
2213             NaN                                  NaN
...              ...                                  ...
2336             NaN   University of the West of Scotland
2337             NaN                University of Windsor
2338             NaN          University of Wolverhampton
2339             NaN              University of Wuppertal
2340             NaN  Xi’an Jiaotong-Liverpool University

[132 rows x 2 columns]

Hay 132 filas con NaN en la columna 'University Rank'
Total de registros después de limpiar: 2209


### Problemas Estructurales:

1. Lo primero es que la columna "University Rank" contiene multiples valores ya que hay universidades cuyo ranking está entre dos valores, para solucionarlo se descompone en dos columnas: rank_min y rank_max, para las observaciones que tienen el valor "Reporter" se le asignará NaN. Además, existen valores como "1501+" que indican que el ranking es igual o mayor a 1501, en estos casos, el valor asignado a rank_max será el número máximo de universidades presentes en el ranking. Aunque se busca que los datos sean usables, es importante reconocer que esta decisión introduce cierto sesgo, ya que no se cuenta con el valor máximo real del rango para estas universidades.


In [105]:

# Reemplazar 'Reporter' con NaN y normalizar guiones
df['University Rank'] = df['University Rank'].replace('Reporter', np.nan)
df['University Rank'] = df['University Rank'].str.replace('–', '-').str.replace('—', '-')

# Reemplazar "+" por un rango con el máximo
max_rank = len(df)
df['University Rank'] = df['University Rank'].str.replace('+', f'-{max_rank}')

# Dividir por el guión
df[['rank_min', 'rank_max']] = df['University Rank'].str.split('-', expand=True)

# Si rank_max es nulo, usar rank_min
df['rank_max'] = df['rank_max'].fillna(df['rank_min'])

# Convertir a numérico
df['rank_min'] = pd.to_numeric(df['rank_min'], errors='coerce').astype('Int64')
df['rank_max'] = pd.to_numeric(df['rank_max'], errors='coerce').astype('Int64')


# Reordenar columnas
cols = list(df.columns)
cols.insert(1, cols.pop(cols.index('rank_min')))
cols.insert(2, cols.pop(cols.index('rank_max')))
df = df[cols]

print(df[['University Rank', 'rank_min', 'rank_max']].sample(10))

df.sample(5)



     University Rank  rank_min  rank_max
1563       1501-2209      1501      2209
107              108       108       108
143              144       144       144
346          301-350       301       350
412          401-500       401       500
1036       1001-1200      1001      1200
977        1001-1200      1001      1200
1864             NaN      <NA>      <NA>
1866             NaN      <NA>      <NA>
1472       1501-2209      1501      2209


,University Rank,rank_min,rank_max,Name of University,Location,No of student,No of student per staff,International Student,Female:Male Ratio,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score
1650,1501-2209,1501,2209,University of Shizuoka,Japan,"3,125",11.9,3%,61 : 39,10.4–18.3,19.5,12.8,10.2,41.5,22.8
2092,NaN,<NA>,<NA>,Satya Wacana Christian University,Indonesia,"16,132",35.5,0%,45 : 55,NaN,NaN,NaN,NaN,NaN,NaN
955,801-1000,801,1000,Yeungnam University,South Korea,"19,345",15.0,4%,50 : 50,29.8–33.9,20.2,15.3,54.0,42.7,43.3
315,301-350,301,350,Florida State University,United States,"39,380",24.0,7%,57 : 43,47.0–48.7,40.2,37.3,67.5,40.1,47.8
1141,1001-1200,1001,1200,University of Trás-os-Montes and Alto Douro,Portugal,"7,123",11.1,8%,55 : 45,24.4–29.7,17.1,19.4,31.9,36.9,44.2



2. El mismo problema ocurre en la columna "OverAll Score" hay registros que representan un valor único y otros que representan un rango, entonces de igual manera se separa en dos columnas: Score_Min y Score_Max, para los nulos o n/a se transforman en NaN.


In [ ]:

# Reemplazar 'n/a' y valores nulos con NaN
df['OverAll Score'] = df['OverAll Score'].replace(['n/a', 'N/A', '-', ' '], np.nan)

# Normalizar diferentes tipos de guiones
df['OverAll Score'] = df['OverAll Score'].str.replace('–', '-').str.replace('—', '-')

# Dividir columna por el guión
df[['Score_Min', 'Score_Max']] = df['OverAll Score'].str.split('-', expand=True)

# Si Score_Max es nulo, usar Score_Min
df['Score_Max'] = df['Score_Max'].fillna(df['Score_Min'])

# Convertir a numérico
df['Score_Min'] = pd.to_numeric(df['Score_Min'], errors='coerce')
df['Score_Max'] = pd.to_numeric(df['Score_Max'], errors='coerce')

# Verificar
print(df[['OverAll Score', 'Score_Min', 'Score_Max']].sample(20))

     OverAll Score  Score_Min  Score_Max
1581     10.4-18.3       10.4       18.3
1345     18.4-24.3       18.4       24.3
1176     18.4-24.3       18.4       24.3
2046           NaN        NaN        NaN
1921           NaN        NaN        NaN
326      47.0-48.7       47.0       48.7
1824           NaN        NaN        NaN
1530     10.4-18.3       10.4       18.3
82            65.3       65.3       65.3
489      42.1-44.9       42.1       44.9
1961           NaN        NaN        NaN
487      42.1-44.9       42.1       44.9
23            82.7       82.7       82.7
943      29.8-33.9       29.8       33.9
97            63.5       63.5       63.5
1989           NaN        NaN        NaN
1431     18.4-24.3       18.4       24.3
284      48.9-51.1       48.9       51.1
1917           NaN        NaN        NaN
1086     24.4-29.7       24.4       29.7



3. En la columna Female:Male Ratio también tiene multiples variables por registro, se debe hacer una limpieza de los espacios y luego descomponer en female_ratio y male_ratio, de esta forma se mejora la usabilidad de la columna y es más facil aplicar técnicas de DM.




In [32]:
# 1. Dividimos la columna usando el separador ':'
df_split = df['Female:Male Ratio'].str.split(':', expand=True)

# 2. Asignamos los resultados a nuevas columnas en tu DataFrame original
# Usamos .str.strip() para eliminar los espacios en blanco que sobran
df['Female_ratio'] = df_split[0].str.strip()
df['Male_ratio'] = df_split[1].str.strip()

# 3. Aseguramos en convertirlos a números 
df['Female_ratio'] = pd.to_numeric(df['Female_ratio'])
df['Male_ratio'] = pd.to_numeric(df['Male_ratio'])

#4. Eliminamos la columna original
df_new=df.drop(columns=['Female:Male Ratio'])

df_new.head()

,University Rank,Name of University,Location,No of student,No of student per staff,International Student,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score,Female_ratio,Male_ratio
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,96.4,92.3,99.7,99.0,74.9,96.2,48.0,52.0
1,2,Harvard University,United States,"21,887",9.6,25%,95.2,94.8,99.0,99.3,49.5,80.5,50.0,50.0
2,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,94.8,90.9,99.5,97.0,54.2,95.8,47.0,53.0
3,3,Stanford University,United States,"16,164",7.1,24%,94.8,94.2,96.7,99.8,65.0,79.8,46.0,54.0
4,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,94.2,90.7,93.6,99.8,90.9,89.3,40.0,60.0


### Problemas Funcionales:

1. Hay registros sin ranking nombre de la universidad, esto introduce ruido al dataset y se debe eliminar.


In [20]:
# Calcular cuántas filas tienen NaN en 'Name of University'
total_nulos = df['Name of University'].isna().sum()
print(f"Hay {total_nulos} filas con NaN en la columna 'Name of University'")

# Eliminar filas donde 'Name of University' está vacío o es NaN
df = df.dropna(subset=['Name of University'])

# También eliminar filas con strings vacíos o solo espacios en blanco
df = df[df['Name of University'].str.strip() != '']

# Verificar cuántos registros quedaron
print(f"Total de registros después de limpiar: {len(df)}")
df.head()

Hay 0 filas con NaN en la columna 'Name of University'
Total de registros después de limpiar: 2233


,University Rank,Name of University,Location,No of student,No of student per staff,International Student,Female:Male Ratio,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,48 : 52,96.4,92.3,99.7,99.0,74.9,96.2
1,2,Harvard University,United States,"21,887",9.6,25%,50 : 50,95.2,94.8,99.0,99.3,49.5,80.5
2,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,47 : 53,94.8,90.9,99.5,97.0,54.2,95.8
3,3,Stanford University,United States,"16,164",7.1,24%,46 : 54,94.8,94.2,96.7,99.8,65.0,79.8
4,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,40 : 60,94.2,90.7,93.6,99.8,90.9,89.3


2. Hay universidades que no tienen información en campos importantes como el ranking, el número de estudiantes y los ratios, lo que nos deja sin información útil para el análisis por lo que se procede a eliminarlas.



3. La columna "International Student" se entiende como una variable de proporción y para evitar mezclar números con el caracter %, se procede a limpiar y normalizar, eliminando el caracter y transformando el campo a tipo númerico.